<a href="https://colab.research.google.com/github/SamigullinRT/HPC/blob/main/lab_2(vector_sum)/vector_sum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5jykbrlg
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5jykbrlg
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=a2c74c29ab1574d86f3000fd8f5612db451c2f1bd25becbcc664fe3a93d5cbd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-46xtc6ud/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [11]:
%%cu
#include <iostream>
#include <vector>
#include <chrono>

// Функция сложения элементов вектора на CPU
int sumVectorCPU(const std::vector<int>& vec) {
    int sum = 0;
    for (int i = 0; i < vec.size(); i++) {
        sum += vec[i];
    }
    return sum;
}

// Функция сложения элементов вектора на GPU с использованием CUDA
__global__ void sumVectorGPU(const int* vec, int* sum, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < size) {
        atomicAdd(sum, vec[tid]);
    }
}

int main() {
    // Создание и заполнение вектора
    const int size = 10e+8;
    std::vector<int> vec(size);
    for (int i = 0; i < size; i++) {
        vec[i] = i + 1;
    }

    // Сложение элементов на CPU
    auto startCPU = std::chrono::high_resolution_clock::now();
    int sumCPU = sumVectorCPU(vec);
    auto endCPU = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> cpuDuration = endCPU - startCPU;

    // Сложение элементов на GPU с использованием CUDA
    int* devVec;
    int* devSum;
    cudaMalloc((void**)&devVec, size * sizeof(int));
    cudaMalloc((void**)&devSum, sizeof(int));
    cudaMemcpy(devVec, vec.data(), size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemset(devSum, 0, sizeof(int));

    int threadsPerBlock = 256;
    int blocksPerGrid = (size + threadsPerBlock - 1) / threadsPerBlock;

    auto startGPU = std::chrono::high_resolution_clock::now();
    sumVectorGPU <<<blocksPerGrid, threadsPerBlock>>> (devVec, devSum, size);
    cudaDeviceSynchronize();
    int sumGPU;
    cudaMemcpy(&sumGPU, devSum, sizeof(int), cudaMemcpyDeviceToHost);
    auto endGPU = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> gpuDuration = endGPU - startGPU;

    // Освобождение памяти на GPU
    cudaFree(devVec);
    cudaFree(devSum);

    // Вывод результатов
    std::cout << "Сумма элементов на CPU: " << sumCPU << std::endl;
    std::cout << "Время выполнения на CPU: " << cpuDuration.count() << " сек" << std::endl;
    std::cout << "Сумма элементов на GPU: " << sumGPU << std::endl;
    std::cout << "Время выполнения на GPU: " << gpuDuration.count() << " сек" << std::endl;

    return 0;
}

Сумма элементов на CPU: -243309312
Время выполнения на CPU: 4.93901 сек
Сумма элементов на GPU: -243309312
Время выполнения на GPU: 0.0484628 сек

